In [8]:
import numpy as np
import pandas as pd
from rdflib.util import more_than
from datetime import datetime
file_path = "../Dataset/restaurant_orders.csv"
ddf = pd.read_csv(file_path)

df = ddf.copy()
df['final'] = df['Price'] * df['Quantity']
df['Customer count'] = df.groupby('Customer Name')['Customer Name'].transform('count')
df


,Order ID,Customer Name,Food Item,Category,Quantity,Price,Payment Method,Order Time,final,Customer count
0,2268,Mary Vega DDS,Pasta,Main,5,16.52,Cash,2025-02-02 14:28:41,82.60,1
1,3082,Brandon Myers,Brownie,Dessert,4,17.27,Debit Card,2025-06-08 10:57:47,69.08,1
2,3160,Margaret Wells,Pasta,Main,1,3.37,Credit Card,2025-03-04 07:41:41,3.37,1
3,1272,Michael Matthews,Pasta,Main,5,2.20,Online Payment,2025-05-15 12:43:45,11.00,1
4,9447,Connor Williams,Soup,Starter,1,12.23,Cash,2025-03-15 14:25:56,12.23,1
...,...,...,...,...,...,...,...,...,...,...
495,6323,Alyssa Anthony,Pizza,Main,1,21.31,Cash,2025-01-15 19:21:02,21.31,1
496,9836,Jerry Pineda,Soup,Starter,3,15.99,Debit Card,2025-07-15 15:00:19,47.97,1
497,1202,Brandy Smith,Pasta,Main,2,8.54,Credit Card,2025-08-03 23:47:28,17.08,1
498,7876,Ivan Haynes,Soup,Starter,5,20.54,Credit Card,2025-07-23 08:10:06,102.70,1


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Order ID        500 non-null    int64  
 1   Customer Name   500 non-null    object 
 2   Food Item       500 non-null    object 
 3   Category        500 non-null    object 
 4   Quantity        500 non-null    int64  
 5   Price           500 non-null    float64
 6   Payment Method  500 non-null    object 
 7   Order Time      500 non-null    object 
 8   final           500 non-null    float64
 9   Customer count  500 non-null    int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 39.2+ KB


In [17]:
from datetime import datetime
df['hour'] = df['Order Time'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S").hour)
df

,Order ID,Customer Name,Food Item,Category,Quantity,Price,Payment Method,Order Time,final,Customer count,hour
0,2268,Mary Vega DDS,Pasta,Main,5,16.52,Cash,2025-02-02 14:28:41,82.60,1,14
1,3082,Brandon Myers,Brownie,Dessert,4,17.27,Debit Card,2025-06-08 10:57:47,69.08,1,10
2,3160,Margaret Wells,Pasta,Main,1,3.37,Credit Card,2025-03-04 07:41:41,3.37,1,7
3,1272,Michael Matthews,Pasta,Main,5,2.20,Online Payment,2025-05-15 12:43:45,11.00,1,12
4,9447,Connor Williams,Soup,Starter,1,12.23,Cash,2025-03-15 14:25:56,12.23,1,14
...,...,...,...,...,...,...,...,...,...,...,...
495,6323,Alyssa Anthony,Pizza,Main,1,21.31,Cash,2025-01-15 19:21:02,21.31,1,19
496,9836,Jerry Pineda,Soup,Starter,3,15.99,Debit Card,2025-07-15 15:00:19,47.97,1,15
497,1202,Brandy Smith,Pasta,Main,2,8.54,Credit Card,2025-08-03 23:47:28,17.08,1,23
498,7876,Ivan Haynes,Soup,Starter,5,20.54,Credit Card,2025-07-23 08:10:06,102.70,1,8


In [18]:
def meal_time(hour):
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Noon'
    else:
        return 'Evening'

In [19]:
df['Meal Time'] = df['hour'].apply(meal_time)

In [21]:
df.groupby(['Meal Time', 'Food Item']).size().unstack(fill_value=0)

Food Item,Brownie,Burger,Cake,Fries,Ice Cream,Pasta,Pizza,Salad,Soup
Meal Time,,,,,,,,,
Evening,37,28,31,22,27,24,42,24,21
Morning,15,12,13,19,13,9,15,18,13
Noon,11,11,9,19,12,15,11,13,16
